https://www.blog.pythonlibrary.org/2010/02/03/another-step-by-step-sqlalchemy-tutorial-part-1-of-2/

View db with DB browser for SQLite http://sqlitebrowser.org/

Cheat sheet https://www.pythonsheets.com/notes/python-sqlalchemy.html

In [1]:
import sqlalchemy
print(sqlalchemy.__version__)

1.2.15


In [2]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData, Column, Table, ForeignKey
from sqlalchemy import Integer, String
from sqlalchemy import inspect
from sqlalchemy.sql import select
from sqlalchemy.sql import and_
import pandas as pd

In [3]:
# Using a postgres server
# engine = create_engine('postgresql://localhost/robincole') 

In [5]:
# Alternatively using sqlite
engine = create_engine('sqlite:///tutorial.db') # , echo=True

Next step creates the .db file

In [6]:
metadata = MetaData(bind=engine)
 
users_table = Table('users', metadata,
                    Column('id', Integer, primary_key=True),
                    Column('name', String(40)),
                    Column('age', Integer),
                    Column('password', String),
                    )
 
addresses_table = Table('addresses', metadata,
                        Column('id', Integer, primary_key=True),
                        Column('user_id', None, ForeignKey('users.id')),
                        Column('email_address', String, nullable=False)                            
                        )
 
# create tables in database
metadata.create_all()

In [10]:
inspector = inspect(engine)

# Get table information
inspector.get_table_names()

['addresses', 'users']

In [11]:
# Get column information
inspector.get_columns('users')

[{'name': 'id',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'name',
  'type': VARCHAR(length=40),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'age',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'password',
  'type': VARCHAR(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0}]

### Insert some data

In [12]:
# create an Insert object
ins = users_table.insert()
# add values to the Insert object
new_user = ins.values(name="Joe", age=20, password="pass")
 
# create a database connection
conn = engine.connect()
# add user to database by executing SQL
conn.execute(new_user)

In [13]:
# a connectionless way to Insert a user
ins = users_table.insert()
result = engine.execute(ins, name="Shinji", age=15, password="nihongo")
 
# another connectionless Insert
result = users_table.insert().execute(name="Martha", age=45, password="dingbat")

Insert multiple rows

In [14]:
conn.execute(users_table.insert(), [
    {"name": "Ted", "age":10, "password":"dink"},
    {"name": "Asahina", "age":25, "password":"nippon"},
    {"name": "Evan", "age":40, "password":"macaca"}
])

### Selecting

In [15]:
s = select([users_table])
result = s.execute()
 
for row in result:
    print(row)

(1, 'Joe', 20, 'pass')
(2, 'Shinji', 15, 'nihongo')
(3, 'Martha', 45, 'dingbat')
(4, 'Ted', 10, 'dink')
(5, 'Asahina', 25, 'nippon')
(6, 'Evan', 40, 'macaca')


Get all the results in a list of tuples

In [16]:
conn = engine.connect()
res = conn.execute(s)
rows = res.fetchall()

In [17]:
rows

[(1, 'Joe', 20, 'pass'),
 (2, 'Shinji', 15, 'nihongo'),
 (3, 'Martha', 45, 'dingbat'),
 (4, 'Ted', 10, 'dink'),
 (5, 'Asahina', 25, 'nippon'),
 (6, 'Evan', 40, 'macaca')]

In [18]:
s = select([users_table.c.name, users_table.c.age])
result = conn.execute(s)
for row in result:
    print(row)

('Joe', 20)
('Shinji', 15)
('Martha', 45)
('Ted', 10)
('Asahina', 25)
('Evan', 40)


The following is the equivalent to 
`SELECT * FROM users WHERE id > 3`

In [20]:
s = select([users_table], users_table.c.id > 3)
result = conn.execute(s)
for row in result:
    print(row)

(4, 'Ted', 10, 'dink')
(5, 'Asahina', 25, 'nippon')
(6, 'Evan', 40, 'macaca')


You can use the "and_" module to AND multiple fields together

In [23]:
s = select([users_table]).where(and_(users_table.c.name == 'Martha', users_table.c.age < 25))

result = conn.execute(s)
for row in result:
    print(row)

## Pandas
Lets get the data as a dataframe

In [24]:
sql = "select * from users"
print('Performing query...')
df = pd.read_sql(sql, con=engine)

Performing query...


In [25]:
df

,id,name,age,password
0,1,Joe,20,pass
1,2,Shinji,15,nihongo
2,3,Martha,45,dingbat
3,4,Ted,10,dink
4,5,Asahina,25,nippon
5,6,Evan,40,macaca


In [26]:
df_copy = df.copy()

In [27]:
df_copy

,id,name,age,password
0,1,Joe,20,pass
1,2,Shinji,15,nihongo
2,3,Martha,45,dingbat
3,4,Ted,10,dink
4,5,Asahina,25,nippon
5,6,Evan,40,macaca


In [28]:
new_entry = {}
new_entry['id'] = 6
new_entry['name'] = 'Robin'
new_entry['age'] = 100
new_entry['password'] = 'foo'
new_entry

{'id': 6, 'name': 'Robin', 'age': 100, 'password': 'foo'}

In [29]:
new_entry_s = pd.Series(new_entry) # Create the series from a dic

In [30]:
new_entry_s

id              6
name        Robin
age           100
password      foo
dtype: object

In [31]:
df_copy = df_copy.append(new_entry_s, ignore_index=True)

In [32]:
df_copy

,id,name,age,password
0,1,Joe,20,pass
1,2,Shinji,15,nihongo
2,3,Martha,45,dingbat
3,4,Ted,10,dink
4,5,Asahina,25,nippon
5,6,Evan,40,macaca
6,6,Robin,100,foo


In [33]:
df_copy.to_sql(name='users', con=engine, if_exists='replace', index=False)

In [34]:
pd.read_sql_query("select * from users;", conn)

,id,name,age,password
0,1,Joe,20,pass
1,2,Shinji,15,nihongo
2,3,Martha,45,dingbat
3,4,Ted,10,dink
4,5,Asahina,25,nippon
5,6,Evan,40,macaca
6,6,Robin,100,foo
